In [14]:
!pip install einops

import torch
from torch import nn

from einops import rearrange
from einops.layers.torch import Rearrange
import torchvision
import torchvision.transforms as transforms
!pip install wandb
import wandb

# Transformer Implementation

Feedforward block- just a classic 1-hidden-layer MLP

In [13]:
class FeedForward(nn.Module):
  def __init__(self,dim, hidden_dim):
    super().__init__()
    self.network = nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, hidden_dim),
        nn.GELU(),
        nn.Linear(hidden_dim, dim)
    )
  def forward(self, input):
    return self.network(input)


Attention Block

In [4]:
class Attention(nn.Module):
    def __init__(self, dim, heads, dim_head):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.scale = dim_head ** -0.5
        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim, bias = False)

    def forward(self,input):
        x = self.norm(input)
        qkv = self.to_qkv(x).chunk(3,dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')

        return self.to_out(out)

Transformer block
- Normal (non-masked) attention

In [ ]:
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads, dim_head),
                FeedForward(dim, mlp_dim)
            ]))

    def forward(self, x):
      for (attn, ff) in self.layers:
        x = attn(x) + x
        x = ff(x) + x
      return self.norm(x)



# Testing Initial Implementation
Let's verify the base implemenetation functions correctly before moving to eht vision

In [15]:
wandb.init(project="intial-transformer")


wandb: Currently logged in as: pablosfsanchez. Use `wandb login --relogin` to force relogin


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Use a smaller subset of the dataset
subset_indices = list(range(0, len(trainset), 100))  # Use every 10th sample
trainset_subset = torch.utils.data.Subset(trainset, subset_indices)

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=8, shuffle=True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:11<00:00, 897534.34it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 133613.45it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1268991.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7304650.60it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
model = Transformer(dim=28*28, depth=6, heads=8, dim_head=64, mlp_dim=128)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [17]:
from tqdm import tqdm

# basic training
num_epochs = 10
tqdm_bar = tqdm(range(num_epochs))
for epoch in tqdm_bar:
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.view(inputs.size(0), -1)  # Flatten the input images
        inputs = inputs.unsqueeze(1)  # Add a sequence length dimension (ugly)

        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze(1)  # Remove the sequence length dimension
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    tqdm_bar.set_description(f"Processing {epoch}")

    epoch_loss = running_loss / len(trainloader)
    wandb.log({"loss": epoch_loss}, step=epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

wandb.finish()


Processing 0:  10%|█         | 1/10 [00:09<01:24,  9.35s/it]

Epoch [1/10], Loss: 0.1134


Processing 1:  20%|██        | 2/10 [00:18<01:14,  9.35s/it]

Epoch [2/10], Loss: 0.1196


wandb: WARNING Step only supports monotonically increasing values, use define_metric to set a custom x axis. For details see: https://wandb.me/define-metric
wandb: WARNING (User provided step: 0 is less than current step: 1. Dropping entry: {'loss': 0.11344627781305462, '_timestamp': 1714779050.0310264}).
Processing 2:  30%|███       | 3/10 [00:27<01:03,  9.10s/it]

...

Processing 8:  90%|█████████ | 9/10 [01:23<00:09,  9.26s/it]

Epoch [9/10], Loss: 0.0011


Processing 9: 100%|██████████| 10/10 [01:33<00:00,  9.30s/it]


Epoch [10/10], Loss: 0.0008


loss,▅█▆▃▂▂▁▁▁▁
loss,0.00084


# Vision Transformer

Now to actually make it a vision transformer...

We should add the positional embeddings and use the technique in the paper to flatten the input

Rehsape the image (H x W x C) to flattened 2D patches of (N x (P^2*c))

Then, flatten the patches and map to D dimensions with a trainable linear projection (get the patch embeddings)

We still apply positional embeddings like in the original transformers paper

In [71]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

# Testing
Now time to test the implementation


In [72]:
wandb.init(project="vision-transformer-test")

# Create an instance of the ViT model
model = ViT(
    channels=1,
    image_size=28,
    patch_size=7,
    num_classes=10,
    dim=64,
    depth=6,
    heads=8,
    mlp_dim=128,
    dropout=0.1,
    emb_dropout=0.1
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
tqdm_bar = tqdm(range(num_epochs))
for epoch in tqdm_bar:
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        # print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    tqdm_bar.set_description(f"Processing {epoch}")

    epoch_loss = running_loss / len(trainloader)
    wandb.log({"loss": epoch_loss}, step=epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

wandb.finish()


Processing 0:  10%|█         | 1/10 [00:03<00:29,  3.32s/it]

Epoch [1/10], Loss: 1.7651


Processing 1:  20%|██        | 2/10 [00:06<00:25,  3.23s/it]

Epoch [2/10], Loss: 0.9692


Processing 2:  30%|███       | 3/10 [00:10<00:23,  3.39s/it]

...

Processing 8:  90%|█████████ | 9/10 [00:30<00:03,  3.40s/it]

Epoch [9/10], Loss: 0.1551


Processing 9: 100%|██████████| 10/10 [00:34<00:00,  3.40s/it]


Epoch [10/10], Loss: 0.1460


loss,█▅▃▂▂▂▁▁▁▁
loss,0.14596


# Saving Model

In [61]:
torch.save(model.state_dict(), 'vit_model.pth')

In [63]:
!pip install huggingface_hub

In [66]:
from huggingface_hub import notebook_login
notebook_login()

In [67]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id='pablojosecodes/basic_vision_transformer', exist_ok=True)
api.upload_file(path_or_fileobj='vit_model.pth', path_in_repo='vit_model.pth', repo_id='pablojosecodes/basic_vision_transformer')


vit_model.pth:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pablojosecodes/basic_vision_transformer/commit/a11c10b85889eb249a1cf446f2722642cd11dabb', commit_message='Upload vit_model.pth with huggingface_hub', commit_description='', oid='a11c10b85889eb249a1cf446f2722642cd11dabb', pr_url=None, pr_revision=None, pr_num=None)

# Loading Model
Sample loading the weights from Huggingface

In [70]:
!pip install huggingface_hub
from huggingface_hub import hf_hub_download
import torch

# Download the trained model from the Hugging Face Hub
model_file = hf_hub_download('pablojosecodes/basic_vision_transformer', 'vit_model.pth')

# Create an instance of the ViT model
model2 = ViT(
    channels=1,
    image_size=28,
    patch_size=7,
    num_classes=10,
    dim=64,
    depth=6,
    heads=8,
    mlp_dim=128,
    dropout=0.1,
    emb_dropout=0.1
)

# Load the trained model weights
model2.load_state_dict(torch.load(model_file))


<All keys matched successfully>

In [ ]:
v